In [1]:
# #In lambdalabs jupyter lab instance, run these:
# pip install transformers
# pip install seaborn
# pip install tf-keras
# pip install --upgrade "numpy<2"
# pip install datasets
# pip install --upgrade datasets pillow
# pip install --upgrade "accelerate>=0.26.0"
# #then check dependency warnings
# pip check
# #if any issues run these SEPARATELY!
# pip install debugpy
# pip install --upgrade argcomplete
# # then install these
# sudo apt-get update
# sudo apt-get install python3-cairo

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import seaborn as sns
import subprocess
import tensorflow as tf
import torch
import torchvision.transforms as T
from collections import Counter
from datasets import load_dataset, Image as DatasetsImage
from datasets import ClassLabel, Dataset, Features, concatenate_datasets
from datetime import datetime
from functools import partial
from io import BytesIO
from pathlib import Path
from PIL import Image, ImageOps, ExifTags, UnidentifiedImageError
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn
from torch.nn import functional as F
from torchvision.transforms import ToPILImage
from tqdm import tqdm
from transformers import (
    AutoImageProcessor, 
    AutoModelForImageClassification, 
    EarlyStoppingCallback,
    TrainingArguments, 
    Trainer
)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-04-19 19:51:17.803359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745092277.826473   17789 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745092277.834438   17789 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745092277.853769   17789 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:174509227

In [ ]:
# # --------------------------
# # GPU Environment Setup for Multi-GPU Optimization (GPUs 0-n)
# # --------------------------
# # Limit process to specific GPUs
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1" #0, 1, 2, 3,...n
# print("Process restricted to GPUs:", os.environ["CUDA_VISIBLE_DEVICES"])

# # Ensure pip executables are available
# os.environ["PATH"] = f"{os.path.expanduser('~/.local/bin')}:" + os.environ["PATH"]

# # Enable memory growth for TensorFlow
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         print("Memory growth enabled on GPUs.")
#     except RuntimeError as e:
#         print("Error configuring GPUs:", e)
# print("GPUs available to this process (as seen by TensorFlow):", tf.config.list_physical_devices('GPU'))

# # Optional: Monitor current GPU usage
# gpu_usage = subprocess.check_output(["nvidia-smi"]).decode("utf-8")
# print("Current GPU usage:\n", gpu_usage)

In [2]:
# --------------------------
# 0. Global Configuration
# --------------------------
RUN_INFERENCE = True  # Toggle this off to disable running inference
IMAGE_DIR = "/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset"

In [3]:
# --------------------------
# 1. Load Pretrained Model and Processor
# --------------------------
model_path = "/home/ubuntu/MLexpressionsStorage/vit_final_independent_V6"

# Load base model and processor
model = AutoModelForImageClassification.from_pretrained(model_path)
processor = AutoImageProcessor.from_pretrained(model_path)

# Update classifier for 8 emotion classes
id2label = {
    0: "anger", 1: "disgust", 2: "fear", 3: "happiness",
    4: "sadness", 5: "surprise", 6: "neutral", 7: "questioning"
}
label2id = {v: k for k, v in id2label.items()}

# Replace classification head to match new number of labels
model.classifier = torch.nn.Linear(model.classifier.in_features, len(id2label))
model.config.num_labels = len(id2label)
model.config.id2label = id2label
model.config.label2id = label2id

# Set to evaluation mode
model.eval()

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [4]:
# --------------------------
# 2. Load and Prepare Dataset
# --------------------------
features = Features({
    'image': DatasetsImage(),
    'label': ClassLabel(names=[
        "anger", "disgust", "fear", "happiness",
        "sadness", "surprise", "neutral", "questioning"
    ])
})

dataset = load_dataset(
    "imagefolder",
    data_dir="/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset",
    split="train",
    cache_dir="/tmp/hf_cache",
    features=features  # ✅ manually define all 8 labels
)

label_mapping = {
    'anger': 'anger', 'disgust': 'disgust', 'fear': 'fear', 'happiness': 'happiness',
    'sadness': 'sadness', 'surprise': 'surprise', 'neutral': 'neutral', 'questioning': 'questioning'
}

counter = {"n": 0}

def reconcile_labels(example):
    counter["n"] += 1
    if counter["n"] % 1000 == 0:
        print(f"Processed {counter['n']} images...")
        
    # If the label is already an integer, convert it to a string using the dataset features.
    if isinstance(example["label"], int):
        # Use dataset.features["label"].int2str to get the string label.
        original_label = dataset.features["label"].int2str(example["label"]).strip().lower()
    else:
        original_label = example["label"].strip().lower()

    # Map the lowercased label to the pre-trained model's expected label.
    pretrain_label = label_mapping.get(original_label)
    # If not recognized, mark it for filtering.
    # Covert the mapped label to its corresponding integer.
    example["label"] = label2id[pretrain_label] if pretrain_label is not None else -1
    return example

# Apply reconciliation function to dataset.
dataset = dataset.map(reconcile_labels, desc="Re-labeling dataset", with_indices=False, num_proc=8)
# Filter out any examples that were marked as unrecognized.
dataset = dataset.filter(lambda x: x["label"] != -1)
print("Total examples after filtering:", len(dataset))

Total examples after filtering: 36788


In [5]:
print(dataset.features["label"])

ClassLabel(names=['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral', 'questioning'], id=None)


In [6]:
# --------------------------
# 3. Define Data Augmentation and Preprocessing Transformation
# --------------------------

# Use torchvision transforms for lightweight CPU-based augmentation.
data_augment = T.Compose([
    T.RandomHorizontalFlip(),                # Random horizontal flip
    T.RandomRotation(10),                      # Random rotation within ±10 degrees
    T.ColorJitter(brightness=0.1, contrast=0.1)  # Slight brightness and contrast changes
])

def transform_function(example, processor):
    label = example["label"]

    # Heavier augmentation for rare classes (e.g. Disgust)
    if label == 1:  # Disgust (minority class)
        aug_pipeline = T.Compose([
            T.RandomResizedCrop(224, scale=(0.7, 1.0)),
            T.RandomHorizontalFlip(p=0.7),
            T.RandomRotation(20),
            T.ColorJitter(0.3, 0.3, 0.3, 0.1),
            T.RandomGrayscale(p=0.2)
        ])
    else:
        aug_pipeline = data_augment

    if example["image"].mode != "RGB":
        example["image"] = example["image"].convert("RGB")

    augmented_image = aug_pipeline(example["image"])
    inputs = processor(augmented_image, return_tensors="pt")
    inputs = {k: v.squeeze(0) for k, v in inputs.items()}
    inputs["labels"] = example["label"]
    return inputs

# Map the transformation to every example in the dataset.
dataset = dataset.map(partial(transform_function, processor=processor))

In [7]:
base_path = "/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset"
print(sorted(os.listdir(base_path)))

['.DS_Store', '._.DS_Store', 'anger', 'disgust', 'fear', 'happiness', 'neutral', 'questioning', 'sadness', 'surprise']


In [9]:
print("Full dataset label distribution:", Counter(dataset["label"]))
for label in sorted(os.listdir(base_path)):
    count = len(os.listdir(os.path.join(base_path, label)))
    print(f"{label}: {count} images")

Full dataset label distribution: Counter({3: 8869, 6: 5996, 4: 5968, 0: 4702, 2: 4671, 5: 4491, 7: 1484, 1: 607})


NotADirectoryError: [Errno 20] Not a directory: '/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/.DS_Store'

In [8]:
# --------------------------
# 4. Train-Validation Split
# --------------------------
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]
# # in case above code doesn't run:
# split_dataset = dataset.train_test_split(test_size=0.2, stratify_by_column="label")
# train_dataset = split_dataset["train"]
# eval_dataset = split_dataset["test"]

In [9]:
# --------------------------
# 5. Balance Dataset
# --------------------------
label_target = 4500
balanced_subsets = []

# Dynamically calculate label counts
label_counts = Counter(train_dataset["label"])
print("Original label distribution:", label_counts)

for label, count in label_counts.items():
    subset = train_dataset.filter(lambda x: x['label'] == label, num_proc=8)
    if count > label_target:
        subset = subset.select(random.sample(range(len(subset)), label_target))
    elif count < label_target:
        multiplier = label_target // len(subset)
        remainder = label_target % len(subset)
        subset = concatenate_datasets([subset] * multiplier + [subset.select(range(remainder))])
    balanced_subsets.append(subset)

train_dataset = concatenate_datasets(balanced_subsets).shuffle(seed=42)
print("After balancing:", Counter(train_dataset['label']))

Original label distribution: Counter({3: 7103, 4: 4822, 6: 4796, 0: 3740, 2: 3720, 5: 3579, 7: 1186, 1: 484})


Filter (num_proc=8): 100%|██████████| 29430/29430 [05:03<00:00, 96.95 examples/s] 


After balancing: Counter({1: 4500, 7: 4500, 3: 4500, 0: 4500, 4: 4500, 2: 4500, 5: 4500, 6: 4500})


In [ ]:
# # --------------------------
# # 6 Oversample Underrepresented Classes
# # --------------------------
# def oversample_dataset(dataset):
#     label_counts = Counter(dataset['label'])
#     max_count = max(label_counts.values())
#     label_datasets = []

#     for label in sorted(label_counts):
#         subset = dataset.filter(lambda x: x['label'] == label, num_proc=8)
#         multiplier = max_count // len(subset)
#         remainder = max_count % len(subset)
#         oversampled = concatenate_datasets([subset] * multiplier + [subset.select(range(remainder))])
#         label_datasets.append(oversampled)

#     return concatenate_datasets(label_datasets).shuffle(seed=42)

# train_dataset = oversample_dataset(train_dataset)
# print("After oversampling:", Counter(train_dataset['label']))

In [10]:
# --------------------------
# 7. Define Training Arguments for Robust Fine-Tuning
# --------------------------
training_args = TrainingArguments(
    output_dir="./finetuned_vit_model",    # Directory to save checkpoints and the final model
    eval_strategy="epoch",           # Evaluate at the end of each epoch
    save_strategy="epoch",                 # Save checkpoint at each epoch
    learning_rate=4e-5,                    # A conservative learning rate for fine-tuning
    per_device_train_batch_size=8,         # Adjust based on your CPU memory limits
    per_device_eval_batch_size=8,
    num_train_epochs=5,                    # Fine-tune for a few epochs (adjust as needed)
    load_best_model_at_end=True,           # Automatically load the best model when training finishes
    metric_for_best_model="accuracy",      # Monitor accuracy for best model selection
    logging_dir="./logs",                  # Directory for TensorBoard logs
)

In [11]:
# --------------------------
# 8. Define a Compute Metrics Function for Evaluation
# --------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

# --------------------------
# Confusion Matrix and Per-Class Accuracy Tracking
# --------------------------

# Define a compute_metrics function with confusion matrix logging
def compute_metrics_with_confusion(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    print("\nClassification Report:")
    print(classification_report(labels, preds, target_names=[id2label[i] for i in sorted(id2label)]))

    cm = confusion_matrix(labels, preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm, annot=True, fmt="d", cmap="Blues",
        xticklabels=[id2label[i] for i in sorted(id2label)],
        yticklabels=[id2label[i] for i in sorted(id2label)]
    )
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.tight_layout()
    plt.savefig("confusion_matrix_epoch.png")
    plt.close()

    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

In [12]:
# --------------------------
# 9. Trainer with Class-Weighted Loss
# --------------------------

# Compute class weights from training set
label_freqs = Counter(train_dataset['label'])
total = sum(label_freqs.values())
class_weights = torch.tensor([total / label_freqs[i] for i in range(len(label_freqs))], dtype=torch.float).to("cuda" if torch.cuda.is_available() else "cpu")

# Define custom Trainer to inject class weights
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = F.cross_entropy(logits, labels, weight=class_weights)
        return (loss, outputs) if return_outputs else loss

# trainer initialization
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics_with_confusion,
)

# Fine-tune model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.314000,0.400214,0.879315
2,0.138000,0.495562,0.885431
3,0.058700,0.658850,0.879179
4,0.013200,0.697734,0.890323
5,0.001800,0.694918,0.894265



Classification Report:
              precision    recall  f1-score   support

       anger       0.88      0.86      0.87       962
     disgust       0.96      0.96      0.96       123
        fear       0.86      0.83      0.85       951
   happiness       0.97      0.94      0.96      1766
     sadness       0.83      0.85      0.84      1146
    surprise       0.86      0.86      0.86       912
     neutral       0.88      0.89      0.88      1200
 questioning       0.69      0.81      0.74       298

    accuracy                           0.88      7358
   macro avg       0.87      0.88      0.87      7358
weighted avg       0.88      0.88      0.88      7358


Classification Report:
              precision    recall  f1-score   support

       anger       0.86      0.90      0.88       962
     disgust       1.00      0.95      0.97       123
        fear       0.86      0.84      0.85       951
   happiness       0.94      0.96      0.95      1766
     sadness       0.86      0

TrainOutput(global_step=22500, training_loss=0.11971312008731895, metrics={'train_runtime': 23241.6785, 'train_samples_per_second': 7.745, 'train_steps_per_second': 0.968, 'total_flos': 1.394930822971392e+19, 'train_loss': 0.11971312008731895, 'epoch': 5.0})

In [13]:
# --------------------------
# 10. Save Final Independent Model
# --------------------------
torch.save(model.state_dict(), '/home/ubuntu/MLexpressionsStorage/final_model_V7.pth')
model.save_pretrained("/home/ubuntu/MLexpressionsStorage/vit_final_independent_V7")
processor.save_pretrained("/home/ubuntu/MLexpressionsStorage/vit_final_independent_V7")

['/home/ubuntu/MLexpressionsStorage/vit_final_independent_V7/preprocessor_config.json']

In [14]:
# --------------------------
# 11. Inference Utilities
# --------------------------

# Load model + processor once
model_path = "/home/ubuntu/MLexpressionsStorage/vit_final_independent_V7"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForImageClassification.from_pretrained(model_path).to(device).eval()
processor = AutoImageProcessor.from_pretrained(model_path)
id2label = model.config.id2label

# Single image prediction (unbatched)
def predict_label(image_path, threshold=0.85):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = F.softmax(logits, dim=-1)
        conf, pred_idx = torch.max(probs, dim=-1)
    return (id2label[pred_idx.item()], conf.item()) if conf.item() >= threshold else ("REVIEW", conf.item())

# Batched prediction (for large folders)
def batch_predict(image_folder, batch_size=64, threshold=0.85):
    all_preds = []
    image_paths = [p for p in Path(IMAGE_DIR).rglob("*") if p.suffix.lower() in [".jpg", ".jpeg", ".png", ".tif", ".tiff"]]

    for i in tqdm(range(0, len(image_paths), batch_size), desc="Running inference in batches"):
        batch_paths = image_paths[i:i + batch_size]
        images = []
        valid_paths = []

        for path in batch_paths:
            try:
                img = Image.open(path).convert("RGB")
                images.append(img)
                valid_paths.append(str(path))
            except Exception as e:
                print(f"Error reading {path}: {e}")
                continue

        if not images:
            continue

        inputs = processor(images=images, return_tensors="pt").to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
            probs = torch.nn.functional.softmax(logits, dim=-1)
            confs, preds = torch.max(probs, dim=-1)

        for pred, conf, path in zip(preds.tolist(), confs.tolist(), valid_paths):
            if conf >= threshold:
                all_preds.append(id2label[pred])
            else:
                all_preds.append("REVIEW")  # Flag uncertain cases

    return all_preds

# Distribution plot
def plot_distribution(predictions, output_path):
    label_counts = Counter(predictions)
    labels = sorted(label_counts.keys())
    counts = [label_counts[label] for label in labels]

    plt.figure(figsize=(10, 5))
    plt.bar(labels, counts)
    plt.title("Predicted Expression Distribution")
    plt.xlabel("Expression")
    plt.ylabel("Count")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

In [15]:
# --------------------------
# 12. Entry Point for Inference
# --------------------------
if __name__ == "__main__" and RUN_INFERENCE:
    from datetime import datetime

    OUTPUT_PATH = (
        "/home/ubuntu/MLexpressionsStorage/distribution_plot_V7"
        + datetime.now().strftime("%Y%m%d_%H%M%S")
        + ".png"
    )

    predictions = batch_predict(IMAGE_DIR)
    plot_distribution(predictions, OUTPUT_PATH)
    print(f"Distribution plot saved to: {OUTPUT_PATH}")

Running inference in batches:   0%|          | 0/622 [00:00<?, ?it/s]

Error reading /home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/anger/._PrivateTest_33861977.jpg: cannot identify image file '/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/anger/._PrivateTest_33861977.jpg'


Running inference in batches:  12%|█▏        | 73/622 [00:30<04:01,  2.28it/s]

Error reading /home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/disgust/._Abel_Aguilar_0001.jpg: cannot identify image file '/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/disgust/._Abel_Aguilar_0001.jpg'
Error reading /home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/disgust/._Adrian_Nastase_0001.jpg: cannot identify image file '/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/disgust/._Adrian_Nastase_0001.jpg'
Error reading /home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/disgust/._Al_Gore_0007.jpg: cannot identify image file '/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/disgust/._Al_Gore_0007.jpg'
Error reading /home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/disgust/._Alain_Cervantes_0001.jpg: cannot identify image file '/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/disgust/._Alain_Cervantes_0001.jpg'
Error reading /home/ubuntu/MLexpressionsStorage/

Running inference in batches:  12%|█▏        | 75/622 [00:31<03:09,  2.89it/s]

...

Error reading /home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Roy_Rogers_0001.jpg: cannot identify image file '/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Roy_Rogers_0001.jpg'
Error reading /home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Rubens_Barrichello_0009.jpg: cannot identify image file '/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Rubens_Barrichello_0009.jpg'
Error reading /home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Rubens_Barrichello_0012.jpg: cannot identify image file '/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Rubens_Barrichello_0012.jpg'
Error reading /home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Ruth_Stubbs_0001.jpg: cannot identify image file '/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Ruth_Stubbs_0001.jpg'
Error reading /home/ub

Running inference in batches:  89%|████████▊ | 551/622 [03:56<00:09,  7.53it/s]

Error reading /home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Tim_Salmon_0001.jpg: cannot identify image file '/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Tim_Salmon_0001.jpg'
Error reading /home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Tina_Fey_0001.jpg: cannot identify image file '/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Tina_Fey_0001.jpg'
Error reading /home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Tina_Pisnik_0001.jpg: cannot identify image file '/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Tina_Pisnik_0001.jpg'
Error reading /home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Tippi_Hedren_0001.jpg: cannot identify image file '/home/ubuntu/MLexpressionsStorage/img_datasets/ferckjalf_dataset/surprise/._Tippi_Hedren_0001.jpg'
Error reading /home/ubuntu/MLexpressionsStorage/img_da

Running inference in batches: 100%|██████████| 622/622 [04:26<00:00,  2.33it/s]

Distribution plot saved to: /home/ubuntu/MLexpressionsStorage/distribution_plot_V720250420_031249.png


In [ ]:
# Example usage (uncomment to test):
# label = predict_label("/path/to/image.jpg")
# print("Predicted Label:", label)

In [ ]:
# from transformers import AutoFeatureExtractor, AutoModelForImageClassification
# import torch
# from PIL import Image

# model_name = "nateraw/fake-face-detector"  # or try harish2704/fake-face-detector
# model = AutoModelForImageClassification.from_pretrained(model_name).eval()
# processor = AutoFeatureExtractor.from_pretrained(model_name)

# def is_real_face(image_path, threshold=0.5):
#     image = Image.open(image_path).convert("RGB")
#     inputs = processor(images=image, return_tensors="pt")
#     with torch.no_grad():
#         logits = model(**inputs).logits
#         probs = torch.nn.functional.softmax(logits, dim=-1)
#         fake_prob = probs[0][1].item()  # index 1 is typically 'fake'
#     return fake_prob < threshold  # True if real
